In [ ]:

import ee

# Trigger the authentication flow.
#ee.Authenticate()

# Initialize the library.
ee.Initialize()

import geemap
import json
import matplotlib.pyplot as plt
import numpy as np
from matplotlib.animation import FuncAnimation
from skimage import exposure
import cv2
import os



LANDSAT_5 = "LANDSAT/LT05/C02/T1_L2"
LANDSAT_5_START = 1984
LANDSAT_5_END = 2000

LANDSAT_7 = "LANDSAT/LE07/C02/T1_L2"
LANDSAT_7_START = 2000
LANDSAT_7_END = 2014

LANDSAT_8 = "LANDSAT/LC08/C02/T1_L2"
LANDSAT_8_START = 2014
LANDSAT_8_END = 2022

def apply_scale_factors_57(image):
    opticalBands = image.select('SR_B.').multiply(0.0000275).add(-0.2)
    thermalBand = image.select('ST_B6').multiply(0.00341802).add(149.0)

    image = (image.addBands(opticalBands, None, True)
             .addBands(thermalBand, None, True))
    image = (image.select(["SR_B1", "SR_B2", "SR_B3", "SR_B4", "SR_B5", "SR_B7", "ST_B6", "QA_PIXEL"])
             .rename(["B1", "B2", "B3", "B4", "B5", "B7", "ST", "QA_PIXEL"]))
    return image


def apply_scale_factors_8(image):
    opticalBands = image.select('SR_B.').multiply(0.0000275).add(-0.2)
    thermalBand = image.select('ST_B.*').multiply(0.00341802).add(149.0)

    image = (image.addBands(opticalBands, None, True)
             .addBands(thermalBand, None, True))
    image = (image.select(["SR_B2", "SR_B3", "SR_B4", "SR_B5", "SR_B6", "SR_B7", "ST_B10", "QA_PIXEL"])
             .rename(  ["B1",    "B2",    "B3",    "B4",    "B5",    "B7",    "ST",     "QA_PIXEL"]))
    return image

def mask_clouds(image):
    qa = image.select("QA_PIXEL")

    mask = (qa.bitwiseAnd(1 << 3).And(qa.bitwiseAnd(1 << 9))
            .Or(qa.bitwiseAnd(1 << 4).And(qa.bitwiseAnd(1 << 11)))
            .Or(qa.bitwiseAnd(1 << 5).And(qa.bitwiseAnd(1 << 13)))
            )

    image = image.updateMask(mask.Not())

    return image


image_ls_8 = (ee.ImageCollection(LANDSAT_8)
              .map(apply_scale_factors_8)
              .map(mask_clouds)
              )

COMBINED_LANDSAT = image_ls_8